## Calculating lobe volumes of all CTs

In [ ]:
import os
import csv
import numpy as np
import nibabel as nib
import pandas as pd

# === Input & Output paths
input_folder = r"----- INSERT PATH HERE ------"
output_csv = r"----- INSERT csv PATH HERE ------"

# === Collect all top-level .nii.gz files
nii_files = [f for f in os.listdir(input_folder) if f.endswith(".nii.gz") and os.path.isfile(os.path.join(input_folder, f))]

# === Initialize CSV
with open(output_csv, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["MRN", "RML_vol_mL", "RLL_vol_mL", "LL_vol_mL"])  # Header

    for fname in sorted(nii_files):
        mrn = fname.replace(".nii.gz", "")
        fpath = os.path.join(input_folder, fname)

        try:
            # Load segmentation
            img = nib.load(fpath)
            data = img.get_fdata()
            voxel_volume = np.prod(img.header.get_zooms())  # in mm³

            # Compute volume for each label (1, 2, 3)
            vol_rml = np.sum(data == 1) * voxel_volume / 1000  # mL
            vol_rll = np.sum(data == 2) * voxel_volume / 1000
            vol_ll  = np.sum(data == 3) * voxel_volume / 1000

            writer.writerow([mrn, round(vol_rml, 2), round(vol_rll, 2), round(vol_ll, 2)])
            print(f"✅ {mrn}: RML={vol_rml:.1f}, RLL={vol_rll:.1f}, LL={vol_ll:.1f} mL")

        except Exception as e:
            print(f"❌ Failed to process {fname}: {e}")

print(f"\n📄 Volume CSV written to: {output_csv}")
